In [1]:
import numpy as np
np.random.seed(1019)

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

import xgboost

import sklearn
from sklearn.model_selection import train_test_split 

import sys, os, gc, types
import time
from subprocess import check_output

In [2]:
sys.path.append('./utils')

from training import cv, train
from plotting import plot_importance
from data import Data

In [3]:
root_paths = [
    "/data/kaggle-instacart",
    "/Users/jiayou/Dropbox/珺珺的程序/Kaggle/Instacart",
    "/Users/jiayou/Dropbox/Documents/珺珺的程序/Kaggle/Instacart"
]
root = None
for p in root_paths:
    if os.path.exists(p):
        root = p
        break

In [12]:
name = 'v13-t0'


down_sample = None
folds=5
aug=True
num_searches=5

boosting_rounds = 400
# stopping_rounds = 10


----

# xgb top model training and prediction

In [5]:
train_all = Data.train(down_sample=down_sample, aug=aug)

test_all = Data.test(down_sample=down_sample)

Load orig data starts...


./utils/data.py:118: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[col] = df[col].astype(np.float32)
/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


Load orig data: 78.69s
Load aug data starts...
Loaded 10 aug shards
Loaded 20 aug shards
Loaded 30 aug shards
Loaded 40 aug shards
Loaded 50 aug shards
Loaded 60 aug shards
Load aug data: 172.79s


In [6]:
train_df = train_all.copy()
test = test_all.copy()

bnames = ['v13-b{}'.format(i) for i in range(2)]

# Load data with reorder prob - 6 prediction csv files
# merge reorder prob to train_df
for bname in bnames:
    pred=[]
    for i in range(folds):
        pred.append(pd.read_csv(os.path.join(root, 'stacking', 'val_prediction.{}.cv{}.csv'.format(bname, i))))
        pred[i].rename(columns={'reorder_prob':bname}, inplace=True)

    pred_df = pd.concat([pred[i] for i in range(folds)])
    train_df = train_df.merge(pred_df, on=['order_id','product_id'], how='left')

    # merge reorder prob to test
    pred_t_df = pd.read_csv(os.path.join(root, 'stacking', 'test_prediction.{}.csv'.format(bname)))
    pred_t_df.rename(columns={'reordered':bname}, inplace=True)
    test = test.merge(pred_t_df, on=['order_id','product_id'], how='left')

test.sort_index(axis=1, inplace=True)
dtest = xgboost.DMatrix(test.drop(['eval_set', 'order_id', 'reordered', 'product_id'], axis=1))

In [ ]:
pred_t = None
for i in range(num_searches):
    X_train, X_val, y_train, y_val = train_test_split(
        train_df.drop(['eval_set', 'product_id', 'order_id', 'reordered'], axis=1),
        train_df.reordered,
        test_size=0.01, random_state=np.random.randint(0, 10000))

    X_train.sort_index(axis=1, inplace=True)
    X_val.sort_index(axis=1, inplace=True)

    dtrain = xgboost.DMatrix(X_train, y_train)
    dval = xgboost.DMatrix(X_val, y_val)

    gc.collect()
    print(dtrain.num_row(), dtrain.num_col())
    print(dval.num_row(), dval.num_col())

    xgb_params = {
        "booster"          : "gbtree",
        "tree_method"      : "hist",
        "learning_rate"    : 0.05,
        "max_depth"        : 5,
        "min_child_weight" : 10, # hessian weight
        "subsample"        : 0.7,
        "colsample_bytree" : 0.9,

        "objective"        : "reg:logistic",
        "eval_metric"      : "logloss",

        "min_split_loss"   : 0.7, # ?
        "reg_alpha"        : 2e-05,
        "reg_lambda"       : 10,
        "seed"             : np.random.randint(0, 10000)
    #     "grow_policy"      : ["lossguide"]
    }
    
    print(xgb_params)
    
    bst = train(
        xgb_params, dtrain, num_boost_round=boosting_rounds,
        evals=[(dtrain, 'train'), (dval, 'val')])
#         callbacks=[
#                 xgboost.callback.early_stop(stopping_rounds=stopping_rounds)
#         ])

    _, axes = plt.subplots(nrows=1, ncols=5, figsize=(25,18))
    measures = ['weight', 'gain', 'cover']
    for i in range(3):
        plot_importance(bst, height=1, ax=axes[2*i], importance_type=measures[i], title=measures[i])
    plt.show()


    if pred_t is None:
        pred_t = bst.predict(dtest)
    else:
        pred_t += bst.predict(dtest)

pred_t = pred_t / num_searches

test_result = pd.DataFrame({'order_id':test.order_id,'product_id':test.product_id, 'reordered':pred_t})
test_result.to_csv(os.path.join(root, 'stacking', 'test_prediction.{}.csv'.format(name)), index=False)

20294360 90
204994 90
{'tree_method': 'hist', 'reg_alpha': 2e-05, 'learning_rate': 0.05, 'colsample_bytree': 0.9, 'min_child_weight': 10, 'eval_metric': 'logloss', 'seed': 6584, 'min_split_loss': 0.7, 'booster': 'gbtree', 'reg_lambda': 10, 'max_depth': 5, 'subsample': 0.7, 'objective': 'reg:logistic'}
Round 0 update starts...
Round 0 update: 58.88s
Round 0 eval starts...
Round 0 eval: 0.09s
[0]	train-logloss:0.657863	val-logloss:0.657862
Round 1 update starts...
Round 1 update: 2.38s
Round 1 eval starts...
Round 1 eval: 0.06s
[1]	train-logloss:0.625952	val-logloss:0.625951
Round 2 update starts...
Round 2 update: 2.37s
Round 2 eval starts...
Round 2 eval: 0.05s
[2]	train-logloss:0.596972	val-logloss:0.596971
Round 3 update starts...
Round 3 update: 2.34s
Round 3 eval starts...
Round 3 eval: 0.05s
[3]	train-logloss:0.57057	val-logloss:0.570569
Round 4 update starts...
Round 4 update: 2.36s
Round 4 eval starts...
Round 4 eval: 0.05s
[4]	train-logloss:0.546442	val-logloss:0.54644
Round 5 

Round 56 update starts...
Round 56 update: 2.56s
Round 56 eval starts...
Round 56 eval: 0.05s
[56]	train-logloss:0.2475	val-logloss:0.247443
Round 57 update starts...
Round 57 update: 2.60s
Round 57 eval starts...
Round 57 eval: 0.05s
[57]	train-logloss:0.246963	val-logloss:0.246907
Round 58 update starts...
Round 58 update: 2.56s
Round 58 eval starts...
Round 58 eval: 0.05s
[58]	train-logloss:0.246464	val-logloss:0.246409
Round 59 update starts...
Round 59 update: 2.59s
Round 59 eval starts...
Round 59 eval: 0.05s
[59]	train-logloss:0.245999	val-logloss:0.245945
Round 60 update starts...
Round 60 update: 2.56s
Round 60 eval starts...
Round 60 eval: 0.05s
[60]	train-logloss:0.245568	val-logloss:0.245514
Round 61 update starts...
Round 61 update: 2.60s
Round 61 eval starts...
Round 61 eval: 0.05s
[61]	train-logloss:0.245167	val-logloss:0.245115
Round 62 update starts...
Round 62 update: 2.56s
Round 62 eval starts...
Round 62 eval: 0.05s
[62]	train-logloss:0.244795	val-logloss:0.244744
R

Round 113 update starts...
Round 113 update: 2.17s
Round 113 eval starts...
Round 113 eval: 0.05s
[113]	train-logloss:0.240272	val-logloss:0.240299
Round 114 update starts...
Round 114 update: 2.26s
Round 114 eval starts...
Round 114 eval: 0.05s
[114]	train-logloss:0.240266	val-logloss:0.240294
Round 115 update starts...
Round 115 update: 2.30s
Round 115 eval starts...
Round 115 eval: 0.08s
[115]	train-logloss:0.240261	val-logloss:0.24029
Round 116 update starts...
Round 116 update: 2.23s
Round 116 eval starts...
Round 116 eval: 0.05s
[116]	train-logloss:0.240256	val-logloss:0.240286
Round 117 update starts...
Round 117 update: 2.21s
Round 117 eval starts...
Round 117 eval: 0.05s
[117]	train-logloss:0.240251	val-logloss:0.240284
Round 118 update starts...
Round 118 update: 2.23s
Round 118 eval starts...
Round 118 eval: 0.05s
[118]	train-logloss:0.240247	val-logloss:0.240281
Round 119 update starts...
Round 119 update: 2.34s
Round 119 eval starts...
Round 119 eval: 0.05s
[119]	train-log